# 🧹 02. STAGING TRANSFORMATION

## 🎯 MỤC TIÊU:
- Hiểu cách transform từ RAW → STAGING
- Thực hành data cleaning
- Validate data quality

## 📚 NỘI DUNG:
1. Extract từ RAW
2. Clean & Transform
3. Load vào STAGING
4. Validate kết quả

In [1]:
import sys
sys.path.append('../scripts')

import pandas as pd
import re
from datetime import datetime

from db_connector import DatabaseConnector
from data_cleaner import DataCleaner
from validators import DataValidator
from etl_stg import StagingLayerETL

print("✅ Libraries imported!")

✅ Libraries imported!


## 1. EXTRACT TỪ RAW

In [2]:
print("📤 EXTRACT FROM RAW")
print("="*70)

db = DatabaseConnector()

query = """
SELECT customer_id, customer_name, email, country, signup_date, customer_segment
FROM raw.customers
LIMIT 1000
"""

raw_df = db.read_sql(query)
print(f"\n✅ Extracted {len(raw_df):,} rows from RAW")
print(f"\nOriginal shape: {raw_df.shape}")
display(raw_df.head(10))

2025-12-20 09:26:09,470 - db_connector - INFO - Database connector initialized for data_engineer@postgres
2025-12-20 09:26:09,495 - db_connector - INFO - Query executed, DataFrame shape: (1000, 6)


📤 EXTRACT FROM RAW

✅ Extracted 1,000 rows from RAW

Original shape: (1000, 6)


,customer_id,customer_name,email,country,signup_date,customer_segment
0,1,Connor West,brownjessica@example.org,Jamaica,2025-01-01,Premium
1,2,Scott Pierce,joshuawright@example.org,Australia,2025-01-01,Basic
2,3,John Lewis,novaksara@example.org,Iran,2025-01-01,Premium
3,4,Richard Gibson,martinaaron@example.com,Jamaica,2025-01-01,Basic
4,5,lauren daniels,smoore@example.org,San Marino,2025-01-01,Standard
5,6,Kim Brown,brian97@example.net,Malawi,2025-01-01,Premium
6,7,Cynthia Wilson,yorkcasey@example.org,Tuvalu,2025-01-01,Standard
7,8,kevin mills,bethwilliams@example.org,Vietnam,2025-01-01,Premium
8,9,Michael Evans,james53_at_example.com,Germany,2025-01-01,Premium
9,10,Angel Lewis MD,smithchristine@example.net,Venezuela,2025-01-01,Standard


## 2. PHÂN TÍCH ISSUES

In [3]:
print("🔍 PHÂN TÍCH DATA QUALITY ISSUES")
print("="*70)

print("\n1️⃣ DUPLICATES:")
print(f"  Duplicate customer_ids: {raw_df['customer_id'].duplicated().sum()}")
print(f"  Duplicate emails: {raw_df['email'].duplicated().sum()}")

🔍 PHÂN TÍCH DATA QUALITY ISSUES

1️⃣ DUPLICATES:
  Duplicate customer_ids: 0
  Duplicate emails: 0


In [4]:
print("2️⃣ NULL VALUES:")
print(raw_df.isnull().sum())

2️⃣ NULL VALUES:
customer_id          0
customer_name       19
email                0
country              9
signup_date          0
customer_segment     0
dtype: int64


In [5]:
print("3️⃣ INVALID EMAILS:")

def validate_email(email):
    if pd.isna(email):
        return False
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return bool(re.match(pattern, str(email)))

raw_df['email_valid'] = raw_df['email'].apply(validate_email)
invalid_count = (~raw_df['email_valid']).sum()
print(f"  Invalid emails: {invalid_count}")

if invalid_count > 0:
    display(raw_df[~raw_df['email_valid']][['customer_id', 'email']].head())

3️⃣ INVALID EMAILS:
  Invalid emails: 17


,customer_id,email
8,9,james53_at_example.com
52,53,bzimmerman_at_example.org
53,54,umarshall_at_example.net
86,87,ugibson_at_example.org
107,108,leonardthomas_at_example.net


In [6]:
print("4️⃣ LOWERCASE NAMES:")
lowercase_count = raw_df['customer_name'].notna() & raw_df['customer_name'].str.islower()
print(f"  Lowercase names: {lowercase_count.sum()}")

if lowercase_count.sum() > 0:
    display(raw_df[lowercase_count][['customer_id', 'customer_name']].head())

4️⃣ LOWERCASE NAMES:
  Lowercase names: 99


,customer_id,customer_name
4,5,lauren daniels
7,8,kevin mills
18,19,rita keith
38,39,daniel perry
40,41,sarah ashley


## 3. TRANSFORM - STEP BY STEP

In [7]:
print("🧹 TRANSFORM DATA")
print("="*70)

# Make a copy
df = raw_df.copy()
original_count = len(df)

print(f"\nStarting with: {original_count:,} rows")

🧹 TRANSFORM DATA

Starting with: 1,000 rows


In [8]:
# Step 1: Remove duplicate customer_ids
print("\n1️⃣ Remove duplicate customer_ids...")
df = df.drop_duplicates(subset=['customer_id'], keep='first')
print(f"  Removed: {original_count - len(df)} rows")
print(f"  Remaining: {len(df):,} rows")


1️⃣ Remove duplicate customer_ids...
  Removed: 0 rows
  Remaining: 1,000 rows


In [9]:
# Step 2: Remove duplicate emails
print("\n2️⃣ Remove duplicate emails...")
before = len(df)
df = df.drop_duplicates(subset=['email'], keep='first')
print(f"  Removed: {before - len(df)} rows")
print(f"  Remaining: {len(df):,} rows")


2️⃣ Remove duplicate emails...
  Removed: 0 rows
  Remaining: 1,000 rows


In [10]:
# Step 3: Validate emails
print("\n3️⃣ Remove invalid emails...")
before = len(df)
df['email_valid'] = df['email'].apply(validate_email)
df = df[df['email_valid']].drop(columns=['email_valid'])
print(f"  Removed: {before - len(df)} rows")
print(f"  Remaining: {len(df):,} rows")


3️⃣ Remove invalid emails...
  Removed: 17 rows
  Remaining: 983 rows


In [11]:
# Step 4: Capitalize names
print("\n4️⃣ Capitalize names...")

def capitalize_name(name):
    if pd.isna(name):
        return name
    return ' '.join(word.capitalize() for word in str(name).split())

df['customer_name'] = df['customer_name'].apply(capitalize_name)
print("  ✅ Names capitalized")


4️⃣ Capitalize names...
  ✅ Names capitalized


In [12]:
# Step 5: Handle nulls
print("\n5️⃣ Handle nulls...")
before = len(df)
df = df.dropna(subset=['customer_id', 'email', 'signup_date'])
print(f"  Removed rows with NULL in required fields: {before - len(df)}")

# Fill optional nulls
df['country'] = df['country'].fillna('Unknown')
df['customer_segment'] = df['customer_segment'].fillna('Standard')
print("  ✅ Filled optional nulls")


5️⃣ Handle nulls...
  Removed rows with NULL in required fields: 0
  ✅ Filled optional nulls


In [13]:
# Step 6: Add timestamps
df['created_at'] = datetime.now()
df['updated_at'] = datetime.now()

print(f"\n✅ Transform complete!")
print(f"  Input:  {original_count:,} rows")
print(f"  Output: {len(df):,} rows")
print(f"  Removed: {original_count - len(df):,} rows ({(original_count - len(df))/original_count*100:.1f}%)")


✅ Transform complete!
  Input:  1,000 rows
  Output: 983 rows
  Removed: 17 rows (1.7%)


## 4. VALIDATE CLEANED DATA

In [15]:
print("✅ VALIDATE CLEANED DATA")
print("="*70)

validator = (
    DataValidator(df, "staging_customers")
    .check_no_nulls(['customer_id', 'email', 'signup_date'])
    .check_unique(['customer_id', 'email'])
    .check_email_format('email')
    .check_allowed_values('customer_segment', ['Premium', 'Standard', 'Basic'])
)

validator.print_summary()

2025-12-20 09:27:12,915 - validators - INFO - DataValidator initialized for 'staging_customers' with shape (983, 8)
2025-12-20 09:27:12,919 - validators - INFO - ✅ PASS | no_nulls_customer_id: No nulls in 'customer_id'
2025-12-20 09:27:12,920 - validators - INFO - ✅ PASS | no_nulls_email: No nulls in 'email'
2025-12-20 09:27:12,921 - validators - INFO - ✅ PASS | no_nulls_signup_date: No nulls in 'signup_date'
2025-12-20 09:27:12,923 - validators - INFO - ✅ PASS | unique_customer_id: All values unique in 'customer_id'
2025-12-20 09:27:12,924 - validators - INFO - ✅ PASS | unique_email: All values unique in 'email'
2025-12-20 09:27:12,926 - validators - INFO - ✅ PASS | regex_email_email_format: All values match email_format
2025-12-20 09:27:12,928 - validators - INFO - ✅ PASS | allowed_values_customer_segment: All values in allowed list


✅ VALIDATE CLEANED DATA

📊 VALIDATION SUMMARY: staging_customers
Total Rows: 983
Total Rules: 7
✅ Passed: 7
❌ Failed: 0
Pass Rate: 100.0%
Total Failed Rows: 0



## 5. PREVIEW CLEANED DATA

In [16]:
print("👀 PREVIEW CLEANED DATA")
print("="*70)

display(df.head(10))

👀 PREVIEW CLEANED DATA


,customer_id,customer_name,email,country,signup_date,customer_segment,created_at,updated_at
0,1,Connor West,brownjessica@example.org,Jamaica,2025-01-01,Premium,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570
1,2,Scott Pierce,joshuawright@example.org,Australia,2025-01-01,Basic,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570
2,3,John Lewis,novaksara@example.org,Iran,2025-01-01,Premium,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570
3,4,Richard Gibson,martinaaron@example.com,Jamaica,2025-01-01,Basic,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570
4,5,Lauren Daniels,smoore@example.org,San Marino,2025-01-01,Standard,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570
5,6,Kim Brown,brian97@example.net,Malawi,2025-01-01,Premium,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570
6,7,Cynthia Wilson,yorkcasey@example.org,Tuvalu,2025-01-01,Standard,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570
7,8,Kevin Mills,bethwilliams@example.org,Vietnam,2025-01-01,Premium,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570
9,10,Angel Lewis Md,smithchristine@example.net,Venezuela,2025-01-01,Standard,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570
10,11,Roy Warner,nicole35@example.com,New Zealand,2025-01-01,Standard,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570


In [17]:
print("\n📊 Summary Statistics:")
display(df.describe(include='all'))


📊 Summary Statistics:


,customer_id,customer_name,email,country,signup_date,customer_segment,created_at,updated_at
count,983.000000,964,983,983,983,983,983,983
unique,NaN,941,983,238,33,3,NaN,NaN
top,NaN,Bryan Sutton,brownjessica@example.org,Korea,2025-01-18,Standard,NaN,NaN
freq,NaN,2,1,14,50,338,NaN,NaN
mean,503.592065,NaN,NaN,NaN,NaN,NaN,2025-12-20 09:26:28.907697,2025-12-20 09:26:28.913569
min,1.000000,NaN,NaN,NaN,NaN,NaN,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570
25%,255.500000,NaN,NaN,NaN,NaN,NaN,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570
50%,505.000000,NaN,NaN,NaN,NaN,NaN,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570
75%,753.500000,NaN,NaN,NaN,NaN,NaN,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570
max,1000.000000,NaN,NaN,NaN,NaN,NaN,2025-12-20 09:26:28.907698,2025-12-20 09:26:28.913570


## 6. RUN FULL STAGING ETL

In [18]:
print("🚀 RUN FULL STAGING ETL")
print("="*70)

etl_stg = StagingLayerETL(db)

# Run transformation for all tables
results = etl_stg.transform_all()

print(f"\n✅ Staging ETL Complete!")
for table, result in results.items():
    print(f"\n{table}:")
    print(f"  Rows loaded: {result['rows']:,}")

2025-12-20 09:27:17,189 - etl_stg - INFO - ============================================================
2025-12-20 09:27:17,192 - etl_stg - INFO - ETL STAGING LAYER - STARTING
2025-12-20 09:27:17,192 - etl_stg - INFO - ============================================================
2025-12-20 09:27:17,193 - etl_stg - INFO - Transforming customers...
2025-12-20 09:27:17,249 - db_connector - INFO - Query executed, DataFrame shape: (11116, 6)
2025-12-20 09:27:17,356 - db_connector - INFO - Query executed successfully
2025-12-20 09:27:17,358 - db_connector - INFO - Truncated: staging.customers


🚀 RUN FULL STAGING ETL


2025-12-20 09:27:17,931 - db_connector - INFO - Written 10680 rows to staging.customers
2025-12-20 09:27:17,932 - etl_stg - INFO - Customers: 11116 raw -> 10680 stg
2025-12-20 09:27:17,932 - etl_stg - INFO -   Duplicates removed: 0
2025-12-20 09:27:17,933 - etl_stg - INFO -   Invalid emails removed: 224
2025-12-20 09:27:17,934 - etl_stg - INFO -   Nulls removed: 436
2025-12-20 09:27:17,936 - etl_stg - INFO - Transforming products...
2025-12-20 09:27:18,012 - db_connector - INFO - Query executed, DataFrame shape: (100, 5)
2025-12-20 09:27:18,034 - db_connector - INFO - Query executed successfully
2025-12-20 09:27:18,036 - db_connector - INFO - Truncated: staging.products
2025-12-20 09:27:18,050 - db_connector - INFO - Written 100 rows to staging.products
2025-12-20 09:27:18,052 - etl_stg - INFO - Products: 100 raw -> 100 stg
2025-12-20 09:27:18,054 - etl_stg - INFO - Transforming orders...
2025-12-20 09:27:18,069 - db_connector - INFO - Query executed, DataFrame shape: (10680, 1)
2025-1


✅ Staging ETL Complete!

customers:
  Rows loaded: 10,680

products:
  Rows loaded: 100

orders:
  Rows loaded: 110,791

order_items:
  Rows loaded: 333,217


## 7. VERIFY STAGING DATA

In [19]:
print("🔍 VERIFY STAGING DATA")
print("="*70)

verify_query = """
SELECT 
    customer_id,
    customer_name,
    email,
    country,
    customer_segment,
    signup_date,
    created_at
FROM staging.customers
ORDER BY customer_id
LIMIT 10
"""

staging_df = db.read_sql(verify_query)
display(staging_df)

2025-12-20 09:27:48,344 - db_connector - INFO - Query executed, DataFrame shape: (10, 7)


🔍 VERIFY STAGING DATA


,customer_id,customer_name,email,country,customer_segment,signup_date,created_at
0,1,Connor West,brownjessica@example.org,Jamaica,Premium,2025-01-01,2025-12-20 09:27:17.293706
1,2,Scott Pierce,joshuawright@example.org,Australia,Basic,2025-01-01,2025-12-20 09:27:17.293706
2,3,John Lewis,novaksara@example.org,Iran,Premium,2025-01-01,2025-12-20 09:27:17.293706
3,4,Richard Gibson,martinaaron@example.com,Jamaica,Basic,2025-01-01,2025-12-20 09:27:17.293706
4,5,Lauren Daniels,smoore@example.org,San Marino,Standard,2025-01-01,2025-12-20 09:27:17.293706
5,6,Kim Brown,brian97@example.net,Malawi,Premium,2025-01-01,2025-12-20 09:27:17.293706
6,7,Cynthia Wilson,yorkcasey@example.org,Tuvalu,Standard,2025-01-01,2025-12-20 09:27:17.293706
7,8,Kevin Mills,bethwilliams@example.org,Vietnam,Premium,2025-01-01,2025-12-20 09:27:17.293706
8,10,Angel Lewis Md,smithchristine@example.net,Venezuela,Standard,2025-01-01,2025-12-20 09:27:17.293706
9,11,Roy Warner,nicole35@example.com,New Zealand,Standard,2025-01-01,2025-12-20 09:27:17.293706


In [20]:
# Check constraints
print("\n✅ Constraint Checks:")

check_query = """
SELECT 
    COUNT(*) as total_rows,
    COUNT(DISTINCT customer_id) as unique_ids,
    COUNT(DISTINCT email) as unique_emails,
    COUNT(*) - COUNT(customer_name) as null_names,
    COUNT(*) - COUNT(email) as null_emails
FROM staging.customers
"""

checks = db.read_sql(check_query)
display(checks)

2025-12-20 09:27:48,374 - db_connector - INFO - Query executed, DataFrame shape: (1, 5)



✅ Constraint Checks:


,total_rows,unique_ids,unique_emails,null_names,null_emails
0,10680,10680,10680,0,0


## 8. COMPARE RAW vs STAGING

In [21]:
print("📊 COMPARE RAW vs STAGING")
print("="*70)

compare_query = """
SELECT 
    'RAW' as layer,
    COUNT(*) as total_rows,
    COUNT(DISTINCT customer_id) as unique_ids,
    COUNT(DISTINCT email) as unique_emails,
    COUNT(*) - COUNT(customer_name) as null_names
FROM raw.customers
UNION ALL
SELECT 
    'STAGING' as layer,
    COUNT(*) as total_rows,
    COUNT(DISTINCT customer_id) as unique_ids,
    COUNT(DISTINCT email) as unique_emails,
    COUNT(*) - COUNT(customer_name) as null_names
FROM staging.customers
"""

comparison = db.read_sql(compare_query)
display(comparison)

2025-12-20 09:27:48,418 - db_connector - INFO - Query executed, DataFrame shape: (2, 5)


📊 COMPARE RAW vs STAGING


,layer,total_rows,unique_ids,unique_emails,null_names
0,RAW,11116,11116,11116,214
1,STAGING,10680,10680,10680,0


# 🎓 KEY TAKEAWAYS

## ✅ Staging Transformation Steps:
1. **Remove Duplicates**: By customer_id and email
2. **Validate Format**: Email, phone, etc.
3. **Standardize Text**: Capitalize names
4. **Handle Nulls**: Drop required, fill optional
5. **Add Timestamps**: created_at, updated_at
6. **Enforce Constraints**: Primary keys, foreign keys

## 📊 Data Quality Improvements:
- ✅ No duplicate IDs or emails
- ✅ All emails valid format
- ✅ Names properly capitalized
- ✅ No nulls in required fields
- ✅ Referential integrity maintained

## 🔄 Next Step:
- Open `03_production_aggregation.ipynb` để học cách aggregate metrics

In [22]:
print("\n✅ Staging Transformation Complete!")


✅ Staging Transformation Complete!
